In [53]:
import numpy as np
from PIL import Image
import random as rd
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
import math

In [54]:
#Loading the data_set
image_data=[]
for root_dir_name,_,filenames in os.walk('trainingSet'):
    category=root_dir_name
    for filename in filenames:
        if filename.endswith('.py')==False:
            img=Image.open(os.path.join(root_dir_name, filename))
            if img is not None:
                image_data.append([np.array(img)/255,category[-1]])
def image_shuffler():
    image=[]
    true_value=[]
    rd.shuffle(image_data)
    identity_matrix = np.eye(10).tolist()
    for cnt in range(len(image_data)):
        num=int(image_data[cnt][1])
        image.append(image_data[cnt][0])
        true_value.append(identity_matrix[num])
    return np.array(image),np.array(true_value)

In [55]:
class fl_layer:
    def __init__(self,input_size,output_size):
        self.weights=np.random.rand(input_size,output_size)-0.5
        self.bias=np.random.rand(1,output_size)-0.5

    def forward_pass(self,input):
        self.input=input
        self.output=np.dot(input,self.weights)+self.bias
        return self.output
    
    def backward_pass(self,error,learning_rate):
        dj_dw=np.dot(self.input.T,error)
        dj_db=error
        input_error=np.dot(error, self.weights.T)
        self.weights-=learning_rate*dj_dw
        self.bias-=learning_rate*dj_db
        return input_error
    
class activation_layer:
    def __init__(self,activation_function):
        self.activation_function=activation_function
    def forward_pass(self,input):
        self.input=input
        self.activation_function(self,input)
        return self.activation
    def backward_pass(self,error,learning_rate):
        return self.activation_prime*error
    
#Activation Function
    def sigmoid(self,x):
        self.activation=1/(np.exp(-x)+1)
        self.activation_prime=-np.exp(x)*(self.activation)**2
    def tanh(self,x):
        self.activation= np.tanh(x)
        self.activation_prime=1-np.tanh(x)**2
    def ReLU(self,x):
        self.activation=np.maximum(0.01*x, x)
        self.activation_prime=np.where(x > 0, 1, 0.01)
    def softmax(self,x):
        e_x = np.exp(x - np.max(x))
        self.activation= e_x / np.sum(e_x, axis=1, keepdims=True)
        self.activation_prime=self.activation*(1-self.activation)

class Network:
    def __init__(self):
        self.layers = []
    def add(self,layer):
        self.layers.append(layer)
    def categorical_crossentropy(self,y_true, y_pred):
        print(y_true)
        print(y_pred)
        return -np.sum(y_true * np.log(y_pred))
    def categorical_crossentropy_prime(self,y_true, y_pred):
        return y_pred - y_true
    def loss(self,loss,loss_prime):
        self.loss=loss
        self.loss_prime=loss_prime
    def train(self,input,true_value,learning_rate):
        #forward_pass
        curr_output=input
        count_f=0
        count_b=0
        output=[]
        for layer in self.layers:
            output.append(curr_output)
            if count_f==4:
                #print(output[4],output[2])
                curr_output=np.concatenate((output[4],output[2])).reshape(1,16)
                #print(np.shape(curr_output))
            curr_output=layer.forward_pass(curr_output)    
            count_f+=1
        #backward propogation
        error=[]
        curr_error=self.loss_prime(true_value,curr_output)
        loss=self.loss(true_value,curr_output)
        #print('LOSS',loss)
        for layer in reversed(self.layers):
            #print(np.shape(curr_error))
            error.append(curr_error)
            if count_b==4:
                curr_error=error[4][0][:8].reshape(1,8)
            if count_b==6:
                curr_error=error[4][0][8:16].reshape(1,8)
            curr_error=layer.backward_pass(curr_error,learning_rate)
            count_b+=1
        return loss
    
nn=Network()
nn.add(fl_layer(784,8)) # hiddenlayer_1
nn.add(activation_layer(activation_layer.tanh))
nn.add(fl_layer(8,8)) #hiddent_layer2
nn.add(activation_layer(activation_layer.sigmoid))
nn.add(fl_layer(16,8)) #addition_layer
nn.add(activation_layer(activation_layer.ReLU))
nn.add(fl_layer(8,10)) #output_layer
nn.add(activation_layer(activation_layer.softmax))
            

In [56]:
epochs=100
learning_rate=0.1
nn.loss(nn.categorical_crossentropy,nn.categorical_crossentropy_prime)
for _ in range(epochs):
    x,y=image_shuffler()   
    for i in range(2500):
        X,Y=x[i,:,:],y[i,:]
        X=np.ravel(X)
        
        X.tolist()
        X=np.array([X])

        Y.tolist()
        Y=np.array([Y])
        
        loss=nn.train(X,Y,learning_rate)
        print('loss:',loss,'\n','epochs:',_,i)

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0.07400661 0.11001304 0.08468222 0.04365964 0.08827809 0.05727553
  0.10410118 0.08197164 0.05543853 0.30057352]]
loss: 2.2071564153651027 
 epochs: 0 0
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
[[0.05448972 0.11187882 0.11146859 0.0371669  0.13147468 0.06094886
  0.06793573 0.08774161 0.04458671 0.29230839]]
loss: 2.4333590244241106 
 epochs: 0 1
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
[[0.07400164 0.12038114 0.08586405 0.04391143 0.14026743 0.08114712
  0.10283327 0.11905885 0.06431257 0.16822249]]
loss: 2.274646335444192 
 epochs: 0 2
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
[[0.06403333 0.13279061 0.09774644 0.04859483 0.11770316 0.06474838
  0.08550891 0.08221897 0.06018735 0.24646801]]
loss: 2.498369228634863 
 epochs: 0 3
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
[[0.0490882  0.07858482 0.09632527 0.0241412  0.14677311 0.06847225
  0.14139518 0.13577429 0.08754304 0.17190265]]
loss: 1.956196584251727 
 epochs: 0 4
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
[[0.0567472  0.09860298 

C:\Users\Sumeadh\AppData\Local\Temp\ipykernel_20592\387663975.py:27: RuntimeWarning: overflow encountered in exp
  self.activation=1/(np.exp(-x)+1)
C:\Users\Sumeadh\AppData\Local\Temp\ipykernel_20592\387663975.py:28: RuntimeWarning: overflow encountered in exp
  self.activation_prime=-np.exp(x)*(self.activation)**2


loss: nan 
 epochs: 0 787
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
[[nan nan nan nan nan nan nan nan nan nan]]
loss: nan 
 epochs: 0 788
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[nan nan nan nan nan nan nan nan nan nan]]
loss: nan 
 epochs: 0 789
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[[nan nan nan nan nan nan nan nan nan nan]]
loss: nan 
 epochs: 0 790
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
[[nan nan nan nan nan nan nan nan nan nan]]
loss: nan 
 epochs: 0 791
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
[[nan nan nan nan nan nan nan nan nan nan]]
loss: nan 
 epochs: 0 792
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[[nan nan nan nan nan nan nan nan nan nan]]
loss: nan 
 epochs: 0 793
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
[[nan nan nan nan nan nan nan nan nan nan]]
loss: nan 
 epochs: 0 794
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
[[nan nan nan nan nan nan nan nan nan nan]]
loss: nan 
 epochs: 0 795
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
[[nan nan nan nan nan nan nan nan nan nan]]
loss: nan 
 epochs: 0 796
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[na

KeyboardInterrupt: 